In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import datetime as dt
from sklearn.cluster import KMeans
import random as ran
from sklearn.metrics.pairwise import euclidean_distances as euc
from functools import reduce

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from src import crs
from IPython.display import clear_output

In [4]:
# 1. load_excel

# data/apt_all.xlsx
# data/apt_1.xlsx
# data/apt_3.xlsx
# data/apt_4.xlsx
# data/apt_std_nugin_1_3.xlsx

data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

In [5]:
# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)
m.head()

/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  datas_df[household_name] = household_df[col][3:].to_list()
/Users/formegusto/Desktop/idoformegusto/development/research/what-is-public-fee/src/crs/utils/data_preprocessing.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  month_usage_df[name] = [


,아파트1-101-1002,아파트1-101-104,아파트1-101-1102,아파트1-101-1202,아파트1-101-1302,아파트1-101-1401,아파트1-101-1403,아파트1-101-1502,아파트1-101-1504,아파트1-101-1602,...,아파트1-104-303,아파트1-104-404,아파트1-104-408,아파트1-104-503,아파트1-104-507,아파트1-104-606,아파트1-104-705,아파트1-104-708,아파트1-104-902,아파트1-104-905
month,,,,,,,,,,,,,,,,,,,,,
1,253,689,183,239,455,343,380,219,389,350,...,10,111,39,151,181,180,290,184,176,148
2,251,481,177,218,397,297,339,197,348,288,...,77,107,30,114,151,151,263,177,158,127
3,255,424,203,222,378,303,349,202,357,368,...,77,120,28,116,163,157,281,182,171,127
4,252,355,180,230,368,278,298,188,346,365,...,76,64,58,109,151,148,257,175,140,121
5,230,357,258,203,414,270,418,156,320,265,...,150,63,75,137,115,13,84,149,36,117


# 단일계약상의 공동설비사용요금은 어떻게 구성이 되는가?

## 1. 전기요금계의 기본구조
- 전기요금계는 기본요금, 전력량요금, 환경 기후금, 연료비조정액, 필수사용량 보장공제로 이루어진다.
- 이로써 계산된 전기요금계에 부가가치세와 전력기반사업기금이 적용되어 최종청구금액이 산정된다.

In [63]:
_month = 1

month_df = pd.DataFrame(m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

PUBLIC_PERCENTAGE = 10
APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))

crs.utils.print_apt_info(_month, month_df, APT)

APT : 34,560kWh (평균 사용량: 320kWh)
		4,142,150원
세대부 : 31,104 (90%)kWh (평균 사용량: 288kWh)
		3,798,980원
공용부 : 3,456 (10%)kWh (평균 사용량: 32kWh)
		343,170원


## 2. 단일계약 전기요금계의 세부적인 구성

### 1. 아파트 전체 사용량의 평균사용량에 대한 요금 산정

In [64]:
apt_pd = pd.DataFrame(columns=['기본요금', '전력량요금', '환경 기후금', '연료비 조정액', 'VAT', '전력산업기반기금', '기본요금 + 전력량요금'])


apt_pd = apt_pd.append(
    pd.Series({
        "기본요금" : apt.basic,
        "전력량요금" : apt.elec_rate,
        "환경 기후금" : apt.env,
        "연료비 조정액" : apt.fuel * -1,
        'VAT': round(calc.elec_bill * 0.1 / len(month_df)),
        '전력산업기반기금': "x",
        '기본요금 + 전력량요금': "x"
    }, name="가구 수 곱 적용 전"))
apt_pd = apt_pd.append(
    pd.Series({
        "기본요금" : apt.basic * len(month_df),
        "전력량요금" : apt.elec_rate * len(month_df),
        "환경 기후금" : apt.env * len(month_df),
        "연료비 조정액" : apt.fuel * len(month_df) * -1,
        'VAT': round(calc.elec_bill * 0.1),
        '전력산업기반기금': round(mt.floor(calc.elec_bill * 0.037 * 0.1) * 10),
        '기본요금 + 전력량요금': apt.basic * len(month_df) + apt.elec_rate * len(month_df)
    }, name="가구 수 곱 적용 후"))

print()
print("1. 단일계약은 아파트 전체 사용량(APT)의 평균사용량 {}kWh에 대한 주택용 고압 요금을 먼저 산정한다.".format(
    apt.kwh
))
print("2. 전기요금계의 기본구성요소인 기본요금, 전력량요금, 환경 기후금, 연료비 조정액에 가구 수({}) 를 곱하여 최종 전기요금계를 산정한다.".format(
    len(month_df)
))

print()

apt_pd


1. 단일계약은 아파트 전체 사용량(APT)의 평균사용량 360kWh에 대한 주택용 고압 요금을 먼저 산정한다.
2. 전기요금계의 기본구성요소인 기본요금, 전력량요금, 환경 기후금, 연료비 조정액에 가구 수(108) 를 곱하여 최종 전기요금계를 산정한다.



,기본요금,전력량요금,환경 기후금,연료비 조정액,VAT,전력산업기반기금,기본요금 + 전력량요금
가구 수 곱 적용 전,1260,37428,1908,-1080,3952,x,x
가구 수 곱 적용 후,136080,4042224,206064,-116640,426773,157900,4178304


### 2. 가구 별 요금 산정

In [65]:
households = calc.households

households_pd = pd.DataFrame(columns=['기본요금', '전력량요금', '환경 기후금', '연료비 조정액', 'VAT', '전력산업기반기금', '기본요금 + 전력량요금'])

for household in households:
    households_pd = households_pd.append(
        pd.Series({
            "기본요금" : household.basic,
            "전력량요금" : household.elec_rate - household.guarantee,
            "환경 기후금" : household.env,
            "연료비 조정액" : household.fuel * -1,
            'VAT': int(household.vat), 
            '전력산업기반기금': int(household.fund),
            '기본요금 + 전력량요금': household.basic + household.elec_rate
        }, name=household.name))

print()
print("1. 가구 별로 주택용 고압 요금제에 의한 각 가구 사용량에 대한 요금 산정을 진행한다.")
print()
households_pd.head()


1. 가구 별로 주택용 고압 요금제에 의한 각 가구 사용량에 대한 요금 산정을 진행한다.



,기본요금,전력량요금,환경 기후금,연료비 조정액,VAT,전력산업기반기금,기본요금 + 전력량요금
아파트1-101-1002,1260,22201,1340,-759,2404,880,23461
아파트1-101-104,6060,103983,3651,-2067,11163,4130,110043
아파트1-101-1102,730,10913,969,-549,1206,440,14143
아파트1-101-1202,1260,20209,1266,-717,2202,810,21469
아파트1-101-1302,6060,54703,2411,-1365,6181,2280,60763


### 3. 가구 별 요금 수거
#### 단일계약 상에서 아파트 전체 사용요금은 "X 가구 수" 의 과정을 거친다. 그렇기 때문에 특정 가구가 적게 사용했든, 많게 사용했든,  아파트 전체 사용요금은 APT 평균 사용량으로 산정된 가격이 각 가구들에게 분배되어있는 구조로 산정된 요금이다. 그래서 실제 각 가구들이 자신의 사용량만큼의 요금을 계산하는 과정을 수거 과정이라고 명칭했다.

In [66]:
analysis_pd = pd.concat([apt_pd.loc['가구 수 곱 적용 후'],households_pd.sum(), apt_pd.loc['가구 수 곱 적용 후'] - households_pd.sum()], axis=1).T
analysis_pd.index = ['APT', '세대부', '공용부']

print()
print("1. 가구 별로 산정된 요금(세대부) 만큼을 아파트 전체 사용요금에서 수거해간다.")
print("2. 이 후 나머지가 공동설비사용요금이 된다.")
print("• 실제 기본요금은 ")
print()

analysis_pd


1. 가구 별로 산정된 요금(세대부) 만큼을 아파트 전체 사용요금에서 수거해간다.
2. 이 후 나머지가 공동설비사용요금이 된다.



,기본요금,전력량요금,환경 기후금,연료비 조정액,VAT,전력산업기반기금,기본요금 + 전력량요금
APT,136080,4042224,206064,-116640,426773,157900,4178304
세대부,197480,3068820,164805,-93312,334205,123150,3316300
공용부,-61400,973404,41259,-23328,92568,34750,862004


## 3. 단일계약 공동설비사용요금의 구성
#### 부제 : 전기요금계의 기본구성으로 보며, 각 구성이 공동설비사용량의 percentage에 따라 어떻게 달라지는 지 확인해보자
##### 실제 공동설비사용요금은 전기요금계의 기본구성으로 계산이 되어지지 않으며, 단지 가구들이 수거해간 후의 나머지가 되는 형태이다. 이는 세대부가 세부적으로 수거해갔을 때의 공동설비사용요금의 전기요금계의 기본구성을 이렇게 띄고 있겠다. 하고 계산한 표로 이루어지는 분석이다.

In [97]:
_month = 1

month_df = pd.DataFrame(m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

PUBLIC_PERCENTAGE = 10
households_kWh = month_df['usage (kWh)'].sum()
APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))

crs.utils.print_apt_info(_month, month_df, APT)
print("\n---------공동설비사용량 {}% 일 때의 요금표---------".format(PUBLIC_PERCENTAGE))
hh_10, anal_10 = crs.utils.get_analysis_df(_month,month_df, APT)
pd.DataFrame(anal_10['기본요금'])

APT : 34,560kWh (평균 사용량: 320kWh)
		4,142,150원
세대부 : 31,104 (90%)kWh (평균 사용량: 288kWh)
		3,798,980원
공용부 : 3,456 (10%)kWh (평균 사용량: 32kWh)
		343,170원

---------공동설비사용량 10% 일 때의 요금표---------


,기본요금
APT,136080
세대부,197480
공용부,-61400
세대부 / APT percentage,145
공용부 / APT percentage,-45


In [98]:
PUBLIC_PERCENTAGE = 20
APT_20 = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))
crs.utils.print_apt_info(_month, month_df, APT_20)

print("\n---------공동설비사용량 {}% 일 때의 요금표---------".format(PUBLIC_PERCENTAGE))
hh_20, anal_20 = crs.utils.get_analysis_df(_month,month_df, APT_20)
pd.DataFrame(anal_20['기본요금'])

APT : 38,880kWh (평균 사용량: 360kWh)
		4,852,400원
세대부 : 31,104 (80%)kWh (평균 사용량: 288kWh)
		3,798,980원
공용부 : 7,776 (20%)kWh (평균 사용량: 72kWh)
		1,053,420원

---------공동설비사용량 20% 일 때의 요금표---------


,기본요금
APT,136080
세대부,197480
공용부,-61400
세대부 / APT percentage,145
공용부 / APT percentage,-45


#### 공동설비사용량 10% -> 20% 증가 시, 각 구성요소들의 변화 및 분석 결과

##### 기본요금
- 변화하지 않았다.
- 기본요금은 요금 산정에 사용되는 사용량의 누진단계에 따라 고정되는 값이기 때문에 아파트 전체 사용량의 평균 사용량인 320kWh는 공동설비사용량이 20%로 늘어나면서 360kWh가 되었는데, 똑같은 누진 2단계이기 때문에 똑같은 값을 보여줬다.

In [99]:
PUBLIC_PERCENTAGE = 30
APT_30 = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))
crs.utils.print_apt_info(_month, month_df, APT_30)

print("\n---------공동설비사용량 {}% 일 때의 요금표---------".format(PUBLIC_PERCENTAGE))
hh_30, anal_30 = crs.utils.get_analysis_df(_month,month_df, APT_30)
pd.DataFrame(anal_30['기본요금'])

APT : 44,434kWh (평균 사용량: 411kWh)
		6,439,540원
세대부 : 31,104 (71%)kWh (평균 사용량: 288kWh)
		3,798,980원
공용부 : 13,330 (29%)kWh (평균 사용량: 123kWh)
		2,640,560원

---------공동설비사용량 30% 일 때의 요금표---------


,기본요금
APT,654480
세대부,197480
공용부,457000
세대부 / APT percentage,30
공용부 / APT percentage,70


이와 같이, 30% 구간에서는 아파트 전체 사용량의 평균 사용량이 411kWh로 요금 산정에 사용되는 사용량의 누진 단계가 3단계이기 때문에 기본요금의 요금이 변화했다.
- 여기서 특이한 점은, 10%와 20% 데이터를 보면, 세대부에서 수거해가는 기본요금이 더 많았지만, 30% 데이터부터는 공용부의 기본요금이 많이 오르면서, 세대부에서 많이 수거해가지 못했다.
##### 이는 아파트 세대부의 가구 사용량 구성에 따라 달라진다.

In [100]:
basic_conf = pd.DataFrame(hh_30['기본요금'].groupby(hh_30['기본요금']).count())
basic_conf.columns = ['가구 수']
basic_conf.reset_index(inplace=True)
basic_conf.index = ["누진 {}단계".format(idx + 1) for idx,_ in enumerate(basic_conf.index)]
basic_conf

,기본요금,가구 수
누진 1단계,730,20
누진 2단계,1260,73
누진 3단계,6060,15


- 해당 아파트의 가구 사용량 구성을 누진단계에 따라 나누어 보면 위와 같다.
- 10%, 20%에서의 APT 평균 사용량은 누진 2단계에 속해있기 때문에 기본요금이 1260원으로 산정된다.
    - 이 때, 기본요금을 수거해가는 가구들 중 730원으로 수거해가는 가구도 있지만, 6060원을 수거해가는 가구도 있다. 1260원과 730원의 차이에 비해, 1260원과 6060원의 차이는 엄청나다. 때문에, 누진 3단계의 가구가 속해 있다면, APT의 평균사용량이 누진 3단계를 기록하지 않는 이상 세대부가 수거해가는 양이 아파트 전체 사용요금 산정의 기본요금보다 많을 것이며, 이로써 공용부 기본요금이 음수(-)의 형태를 나타나게 된다.
- 30%에서의 APT 평균 사용량은 누진 3단계에 속해있기 때문에 기본요금이 6060원으로 산정된다.
    - 6060원을 완전히 수거해 갈 수 있는 가구는 누진 3단계의 가구 밖에 없다. 때문에 누진 3단계의 가구가 많은 아파트가 아니라면, 공용부의 기본요금이 양수(+)의 형태를 나타나게 되고, 누진 3단계의 가구 비율이 낮을 수록 공용부의 기본요금은 상승한다.
    
##### 결론적으로 공용부 사용요금 중, 기본요금에 영향을 주는 요소는 아파트 전체 사용량의 평균 사용량이 가지는 누진단계와 가구들의 구성이다.